In [1]:
# 可视化卷积神经网络过滤器
# 为过滤器的可视化定义损失张量
from keras.applications import VGG16
from keras import backend as k

model = VGG16(
    weights='imagenet',
    include_top = False
)

layer_name = 'block3_conv1'
filter_index = 0

layer_output = model.get_layer(layer_name).output
loss = k.mean(layer_output[:, :, :, filter_index])

Using TensorFlow backend.


ResourceExhaustedError: OOM when allocating tensor with shape[3,3,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add] name: block5_conv1/random_uniform/